In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import  OurRadiologsitsDataset, collate_fn
from our_radiologist.load import get_anns
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ar_ap
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    
    # with_clinical = "val_ar_0_7430_ap_0_5567_test_ar_0_7974_ap_0_5392_epoch29_WithClincal_05-02-2022 17-07-17_with_clinical"

    # with_clinical_final = "val_ar_0_7269_ap_0_4601_test_ar_0_6710_ap_0_4286_epoch100_WithClincal_05-02-2022 19-03-30_with_clinical"

    # with_clinical = "val_ar_0_7430_ap_0_5567_test_ar_0_7974_ap_0_5392_epoch29_WithClincal_05-02-2022 17-07-17_with_clinical"
    # without_clinical = "val_ar_0_5377_ap_0_2981_test_ar_0_4515_ap_0_2716_epoch30_WithoutClincal_05-02-2022 12-32-56_without_clinical"

    # pretrained_resnet50_without_clinical_ap = "val_ar_0_5272_ap_0_2396_test_ar_0_6170_ap_0_1962_epoch44_WithoutClincal_04-19-2022 07-35-55_pretrained_resnet50_without_clinical"
    # pretrained_resnet50_without_clinical_ar = "val_ar_0_5272_ap_0_2396_test_ar_0_6170_ap_0_1962_epoch44_WithoutClincal_04-19-2022 07-35-46_pretrained_resnet50_without_clinical"
    # pretrained_resnet50_without_clinical_final = "val_ar_0_2560_ap_0_1522_test_ar_0_3072_ap_0_1295_epoch200_WithoutClincal_04-20-2022 18-32-16_pretrained_resnet50_without_clinical"

    # pretrained_resnet50_with_clinical_ap = "val_ar_0_3842_ap_0_2467_test_ar_0_3799_ap_0_1727_epoch106_WithClincal_04-20-2022 19-09-13_pretrained_resnet50_with_clinical"
    # pretrained_resnet50_with_clinical_ar = "val_ar_0_5164_ap_0_1869_test_ar_0_5086_ap_0_1718_epoch64_WithClincal_04-19-2022 19-33-35_pretrained_resnet50_with_clinical"
    # pretrained_resnet50_with_clinical_final = "val_ar_0_2849_ap_0_1805_test_ar_0_3793_ap_0_1925_epoch200_WithClincal_04-21-2022 04-07-24_pretrained_resnet50_with_clinical"

    # custom_with_clinical_no_pretrained_ar = "val_ar_0_5220_ap_0_2513_test_ar_0_5590_ap_0_2442_epoch150_WithClincal_04-13-2022 20-13-47_custom_with_clinical_no_pretrained"
    # custom_with_clinical_no_pretrained_ap = "val_ar_0_4554_ap_0_2582_test_ar_0_5254_ap_0_2405_epoch139_WithClincal_04-13-2022 19-05-08_custom_with_clinical_no_pretrained"
    # custom_with_clinical_no_pretrained_final = "val_ar_0_4523_ap_0_2251_test_ar_0_5103_ap_0_2464_epoch200_WithClincal_04-14-2022 01-32-28_custom_with_clinical_no_pretrained"

    # custom_without_clinical_no_pretrained_ar = "val_ar_0_5645_ap_0_2659_test_ar_0_6263_ap_0_2533_epoch145_WithoutClincal_04-13-2022 08-47-34_custom_without_clinical_no_pretrained"
    # custom_without_clinical_no_pretrained_ap = "val_ar_0_5512_ap_0_2962_test_ar_0_5999_ap_0_2319_epoch93_WithoutClincal_04-12-2022 09-15-28_custom_without_clinical_no_pretrained"
    # custom_without_clinical_no_pretrained_final = "val_ar_0_3757_ap_0_1699_test_ar_0_4421_ap_0_1819_epoch200_WithoutClincal_04-13-2022 14-58-52_custom_without_clinical_no_pretrained"

    # custom_without_clinical_swim_ap = "val_ar_0_5307_ap_0_2054_test_ar_0_5321_ap_0_1726_epoch87_WithoutClincal_04-17-2022 06-51-10_custom_without_clinical_swim"
    # custom_without_clinical_swim_ar = "val_ar_0_5313_ap_0_1540_test_ar_0_5906_ap_0_1486_epoch59_WithoutClincal_04-17-2022 05-48-30_custom_without_clinical_swim"
    # custom_without_clinical_swim_final = "val_ar_0_2175_ap_0_1390_test_ar_0_2231_ap_0_0901_epoch200_WithoutClincal_04-17-2022 10-49-59_custom_without_clinical_swim"

    # custom_with_clinical_swim_ap = "val_ar_0_5081_ap_0_2210_test_ar_0_5392_ap_0_1725_epoch95_WithClincal_04-17-2022 15-26-02_custom_with_clinical_swim"
    # custom_with_clinical_swim_ar = "val_ar_0_5377_ap_0_1821_test_ar_0_4561_ap_0_1193_epoch67_WithClincal_04-17-2022 14-02-28_custom_with_clinical_swim"
    # custom_with_clinical_swim_final = "val_ar_0_2752_ap_0_1293_test_ar_0_3391_ap_0_1097_epoch200_WithClincal_04-17-2022 20-31-24_custom_with_clinical_swim"

    # it seems the final two model we will be using.
    # without_clinical = "val_ar_0_5377_ap_0_2981_test_ar_0_4515_ap_0_2716_epoch30_WithoutClincal_05-02-2022 12-32-56_without_clinical"

    # without_clinical_final = "val_ar_0_5053_ap_0_2754_test_ar_0_4265_ap_0_2605_epoch100_WithoutClincal_05-02-2022 14-17-08_without_clinical"

    # with_clinical_lr3_200 = "val_ar_0_7222_ap_0_4626_test_ar_0_6727_ap_0_4956_epoch111_WithClincal_05-03-2022 20-39-41_ov_12"

    # with_clinical_lr3_200_final = "val_ar_0_6650_ap_0_3870_test_ar_0_6604_ap_0_4493_epoch200_WithClincal_05-03-2022 23-04-30_ov_12"


    # CXR_Clinical = "val_ar_0_9556_ap_0_6876_test_ar_0_8438_ap_0_6305_epoch43_WithClincal_05-16-2022 06-58-30_CXR+Clinical"

    # CXR_Clinical_ap = "val_ar_0_9381_ap_0_6945_test_ar_0_8224_ap_0_6378_epoch20_WithClincal_05-16-2022 18-13-10_CXR+Clinical"
    # CXR_Clinical_final = "val_ar_0_9556_ap_0_6776_test_ar_0_7849_ap_0_6534_epoch71_WithClincal_05-16-2022 19-38-03_CXR+Clinical"

    # CXR_ap = "val_ar_0_7518_ap_0_5042_test_ar_0_5388_ap_0_3255_epoch22_WithoutClincal_05-16-2022 20-18-25_CXR"
    # CXR_final = "val_ar_0_5966_ap_0_4046_test_ar_0_5388_ap_0_3893_epoch54_WithoutClincal_05-16-2022 21-10-18_CXR"

    CXR="val_ar_0_4734_ap_0_2989_test_ar_0_3947_ap_0_2774_epoch142_WithoutClincal_05-19-2022 00-56-01_CXR"
    CXR_Clinial="val_ar_0_6160_ap_0_2978_test_ar_0_4945_ap_0_2315_epoch32_WithClincal_05-19-2022 17-45-13_CXR_Clinical"
    


In [4]:
# ========================================For Training [pretrained_resnet50_without_clinical]========================================
# ModelSetup(use_clinical=False, use_custom_model=True, use_early_stop_model=True, name='pretrained_resnet50_without_clinical', best_ar_val_model_path=None, best_ap_val_model_path=None, final_model_path=None, backbone='resnet50', optimiser='sgd', lr=0.0005, weight_decay=5e-05, pretrained=True, record_training_performance=True, dataset_mode='unified')
# ===================================================================================================================================

# Best AP validation model has been saved to: [val_ar_0_5272_ap_0_2396_test_ar_0_6170_ap_0_1962_epoch44_WithoutClincal_04-19-2022 07-35-55_pretrained_resnet50_without_clinical]
# Best AR validation model has been saved to: [val_ar_0_5272_ap_0_2396_test_ar_0_6170_ap_0_1962_epoch44_WithoutClincal_04-19-2022 07-35-46_pretrained_resnet50_without_clinical]
# The final model has been saved to: [val_ar_0_3257_ap_0_1868_test_ar_0_4210_ap_0_1598_epoch100_WithoutClincal_04-19-2022 12-44-29_pretrained_resnet50_without_clinical]

# ===================================================================================================================================


# ========================================For Training [pretrained_resnet50_with_clinical]========================================
# ModelSetup(use_clinical=True, use_custom_model=True, use_early_stop_model=True, name='pretrained_resnet50_with_clinical', best_ar_val_model_path=None, best_ap_val_model_path=None, final_model_path=None, backbone='resnet50', optimiser='sgd', lr=0.0005, weight_decay=5e-05, pretrained=True, record_training_performance=True, dataset_mode='unified')
# ================================================================================================================================

# Best AP validation model has been saved to: [val_ar_0_4246_ap_0_2378_test_ar_0_4963_ap_0_1712_epoch98_WithClincal_04-19-2022 22-54-16_pretrained_resnet50_with_clinical]
# Best AR validation model has been saved to: [val_ar_0_5164_ap_0_1869_test_ar_0_5086_ap_0_1718_epoch64_WithClincal_04-19-2022 19-33-35_pretrained_resnet50_with_clinical]
# The final model has been saved to: [val_ar_0_3271_ap_0_1848_test_ar_0_3900_ap_0_1726_epoch100_WithClincal_04-19-2022 23-06-47_pretrained_resnet50_with_clinical]

# ================================================================================================================================

In [5]:
normal_iou_thrs = iou_thrs_5to95
all_range_iou_thrs = full_iou_thrs
score_thresholds = [0.5, 0.3, 0.2, 0.1, 0.05]

# Run evaluation.

In [6]:
from tqdm import tqdm

In [7]:
for select_model in tqdm(TrainedModels):

    for score_thrs in score_thresholds:

        model, train_info, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

        model.eval()

        iou_types = get_iou_types(model, train_info.model_setup)

        dataset_params_dict = {
            "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
            "with_clinical": train_info.model_setup.use_clinical,
            "dataset_mode": train_info.model_setup.dataset_mode,
            "bbox_to_mask": True,
            "labels_cols": [
                "Enlarged cardiac silhouette",
                "Atelectasis",
                "Pleural abnormality",
                "Consolidation",
                "Pulmonary edema",
                #  'Groundglass opacity', # 6th disease.
            ],
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=4,
        )

        train_coco, val_coco, test_coco = get_cocos(
            train_dataloader, val_dataloader, test_dataloader
        )

        radiologists_ann = get_anns("radiologists_annotated", detect_eval_dataset)

        radiologist_dataset = OurRadiologsitsDataset(detect_eval_dataset, radiologists_ann)
        radiologist_dataloader = torch.utils.data.DataLoader(
            radiologist_dataset,
            batch_size=4,
            shuffle=False,
            collate_fn=collate_fn,
            worker_init_fn=seed_worker,
            generator=get_dataloader_g(0),
        )

        radiologists_coco = get_coco_api_from_dataset(radiologist_dataloader.dataset)

        normal_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=normal_iou_thrs,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )


        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d) for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = xami_evaluate(
                model,
                train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = xami_evaluate(
                model,
                val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            radiologist_evaluator, _ = xami_evaluate(
                model,
                radiologist_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=radiologists_coco,
                iou_types=iou_types,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(train_evaluator, f"train_{disease_str}_score_thrs{score_thrs}", select_model.value)
            save_iou_results(test_evaluator, f"test_{disease_str}_score_thrs{score_thrs}", select_model.value)
            save_iou_results(val_evaluator, f"val_{disease_str}_score_thrs{score_thrs}", select_model.value)
            save_iou_results(
                radiologist_evaluator, f"our_{disease_str}_score_thrs{score_thrs}", select_model.value
            )

            train_ar, train_ap = get_ar_ap(
                train_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            )

            test_ar, test_ap = get_ar_ap(
                test_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            )
            val_ar, val_ap = get_ar_ap(
                val_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            )
            our_ar, our_ap = get_ar_ap(
                radiologist_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            )

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50:0.95]": train_ap,
                        f"AR@[IoBB = 0.50:0.95]": train_ar,
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50:0.95]": test_ap,
                        f"AR@[IoBB = 0.50:0.95]": test_ar,
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50:0.95]": val_ap,
                        f"AR@[IoBB = 0.50:0.95]": val_ar,
                    },
                    {
                        "dataset": "our",
                        f"AP@[IoBB = 0.50:0.95]": our_ap,
                        f"AR@[IoBB = 0.50:0.95]": our_ar,
                    },
                ]
            )

            df.to_csv(
                os.path.join("eval_results", f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv")
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/4 [00:00<?, ?it/s]

Load custom model
Using pretrained backbone. mobilenet_v3
Found optimizer for this model.
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/118]  eta: 0:08:02  loss: 3.0257 (3.0257)  loss_classifier: 3.0091 (3.0091)  loss_box_reg: 0.0166 (0.0166)  loss_objectness: 0.0000 (0.0000)  loss_rpn_box_reg: 0.0000 (0.0000)  model_time: 3.1720 (3.1720)  evaluator_time: 0.0050 (0.0050)  time: 4.0865  data: 0.7115  max mem: 627
Evaluation:  [100/118]  eta: 0:00:16  loss: 1.8834 (2.2204)  loss_classifier: 1.8743 (2.1769)  loss_box_reg: 0.0091 (0.0435)  loss_objectness: 0.0000 (0.0000)  loss_rpn_box_reg: 0.0000 (0.0000)  model_time: 0.0603 (0.0833)  evaluator_time: 0.0015 (0.0015)  time: 0.9790  data: 0.8342  max mem: 1170
Evaluation:  [117/118]  eta: 0:00:00  loss: 1.7044 (2.1736)  loss_classifier: 

 25%|██▌       | 1/4 [1:24:16<4:12:50, 5056.97s/it]

  dataset  AP@[IoBB = 0.50:0.95]  AR@[IoBB = 0.50:0.95]
0   train               0.225876                0.49802
1    test               0.569866                0.75000
2     val               0.855687                1.00000
3     our               0.128713                0.25000
Load custom model
Using pretrained backbone. mobilenet_v3
Found optimizer for this model.
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/118]  eta: 0:01:58  loss: 2.8595 (2.8595)  loss_classifier: 2.8493 (2.8493)  loss_box_reg: 0.0101 (0.0101)  loss_objectness: 0.0000 (0.0000)  loss_rpn_box_reg: 0.0000 (0.0000)  model_time: 0.0500 (0.0500)  evaluator_time: 0.0020 (0.0020)  time: 1.0012  data: 0.7292  max mem: 1170
Evaluation:  [100/118]  eta: 0:00:14  loss: 1.4084 (1.5573)  loss_classifier: 1.4036 (1.5282)  l

 50%|█████     | 2/4 [2:46:41<2:46:21, 4990.89s/it]

  dataset  AP@[IoBB = 0.50:0.95]  AR@[IoBB = 0.50:0.95]
0   train               0.487545               0.694059
1    test               0.572369               0.750000
2     val               0.901345               1.000000
3     our               0.244478               0.500000
Load custom model
Using pretrained backbone. mobilenet_v3
Found optimizer for this model.
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/118]  eta: 0:02:05  loss: 0.4073 (0.4073)  loss_classifier: 0.3980 (0.3980)  loss_box_reg: 0.0093 (0.0093)  loss_objectness: 0.0000 (0.0000)  loss_rpn_box_reg: 0.0000 (0.0000)  model_time: 0.0640 (0.0640)  evaluator_time: 0.0020 (0.0020)  time: 1.0622  data: 0.7472  max mem: 1170
Evaluation:  [100/118]  eta: 0:00:14  loss: 0.2427 (0.3526)  loss_classifier: 0.1866 (0.2898)  l

 75%|███████▌  | 3/4 [4:08:31<1:22:33, 4953.77s/it]

  dataset  AP@[IoBB = 0.50:0.95]  AR@[IoBB = 0.50:0.95]
0   train               0.350453               0.558416
1    test               0.353155               0.587500
2     val               0.563199               0.823077
3     our               0.000000               0.000000
Load custom model
Using pretrained backbone. mobilenet_v3
Found optimizer for this model.
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/118]  eta: 0:02:03  loss: 0.3040 (0.3040)  loss_classifier: 0.2920 (0.2920)  loss_box_reg: 0.0120 (0.0120)  loss_objectness: 0.0000 (0.0000)  loss_rpn_box_reg: 0.0000 (0.0000)  model_time: 0.0490 (0.0490)  evaluator_time: 0.0030 (0.0030)  time: 1.0482  data: 0.6842  max mem: 1170
Evaluation:  [100/118]  eta: 0:00:13  loss: 0.2005 (0.2391)  loss_classifier: 0.1617 (0.1726)  l

100%|██████████| 4/4 [5:30:22<00:00, 4955.69s/it]  

  dataset  AP@[IoBB = 0.50:0.95]  AR@[IoBB = 0.50:0.95]
0   train               0.520153               0.669307
1    test               0.410437               0.493750
2     val               0.307111               0.538462
3     our               0.000000               0.000000
